[Max Woolf](https://github.com/minimaxir/gpt-2-simple)



In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 269Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 111Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 564Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:12, 39.4Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 235Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 166Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 168Mit/s]                                                       


In [4]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [6]:
file_name = "for_train.txt"
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [7]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=2000,
              restore_from='fresh',
              run_name='run1',
              overwrite=True,
              print_every=50,
              sample_every=500,
              save_every=500)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:05<00:00,  5.34s/it]


dataset has 1090522 tokens
Training...
[50 | 47.39] loss=2.46 avg=2.46
[100 | 88.76] loss=2.37 avg=2.42
[150 | 130.17] loss=2.43 avg=2.42
[200 | 171.41] loss=2.48 avg=2.44
[250 | 212.69] loss=2.52 avg=2.46
[300 | 254.21] loss=2.50 avg=2.46
[350 | 295.54] loss=2.21 avg=2.43
[400 | 336.76] loss=2.15 avg=2.39
[450 | 378.10] loss=2.20 avg=2.37
[500 | 419.41] loss=2.44 avg=2.37
Saving checkpoint/run1/model-500
======== SAMPLE 1 ========
 of the new toys. (Gets up) Oh, oh, oh, oh-ohs, oh, oh, oh-ohs.
ELAINE: Well... (Kramer approaches to give her hand a good shake)
KRAMER: Hey, look, look. Jerry, look at me. Do you know what we did in Season 4?
JERRY: You don't think?...
KRAMER: I had a friend say he's gonna let everyone know that he's a friend of mine and that my wife is in love with him... (Elaine starts laughing)
ELAINE: (To Kramer) You know, you must have... Kramer and Elaine
ELAINE: Yes!!! (Kramer gets up to leave.) (Scene ends) (Jerry's car)
JERRY: So? I guess the guy's a man...
GEORGE

In [32]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [31]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [7]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-2000


In [8]:
gpt2.generate(sess, run_name='run1')

BETTE: (Standing up) Well, I have the look of a successful restaurateur, restaurateur, it's unbelievable!
JERRY: What look?
ELAINE: I mean, look at this. He's got a successful restaurant, a successful business... (The phone rings. Jerry answers.)
JERRY: Yeah?
ELAINE: ...ahhh... haaaaaaaaaaaa... (hangs up) This is Elaine Benes, you got the looks!
JERRY: Yeah, right!
ELAINE: Well, I mean, look, I have no clothes, (hands the phone to Jerry) I'm going shopping for a new pair of pants.
JERRY: A new pair of pants?
ELAINE: Yeah, and this looks exactly like Elaine Benes.
JERRY: So you don't know anything?
ELAINE: No, I don't.
JERRY: Like what?
ELAINE: Like I'm a mammal.
JERRY: I don't know...
ELAINE: What? What are you talking about?
JERRY: I don't know...
ELAINE: What about? Jerry; Listen, Elaine, I appreciate it that you're here, but I just wanted to ask you something, like if this relationship is bad, horrible, what is the difference, what's the difference, what am I supposed to do?
JERRY: 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [9]:
gpt2.generate(sess,
              length=400,
              temperature=0.9,
              prefix="SOUP NAZI",
              nsamples=10,
              batch_size=5
              )

SOUP NAZI) Whatever. Peggy returns to her seat, still wearing her Uncle Leo's pained expression.
PEGGY: Do you want me to eat your soup? Giggity, nasty son of a bitch. Jerry and Elaine are at the coffee shop.
JERRY: So I didn't order any.
ELAINE: Yeah, but I wanted the "You can't eat this" bumper sticker. Peggy walks out, her Uncle Leo has gone into reverse work, and Jerry is still wearing his Chinese sippy cup.
JERRY: Hey, I know I'm not the only one unhappy with your eating style. Jerry has gained the upper hand in the shop, with a few customers even yelling at his overbearing behavior.
JERRY: I don't like eating your Uncle Leo's pouty face.
ELAINE: You think you're so original, with your looks, mind you, that nobody takes your Uncle's advice and acting? Jerry and Elaine are still at the mug. Leo opens it.
UNCLE LEO: What is this, a lollipop?
ELAINE: Let's make a tea cup.
JERRY: Ah.
UNCLE LEO: Hey. Peggy picks up the mug and signals Jerry that he has to get in there.
JERRY: Hey. Leo'

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [29]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=400,
                      prefix="SUSAN",
                      temperature=1,
                      nsamples=100,
                      batch_size=20
                      )

In [30]:
# may have to run twice to get file to download
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.